In [ ]:
import pyabf
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
%matplotlib widget

# ==== Load data ====
file_path = 'bursting/cell89basal.abf'
abf = pyabf.ABF(file_path)

# Concatenate sweeps into a single continuous vector
signal = np.concatenate([abf.setSweep(i) or abf.sweepY for i in range(abf.sweepCount)])
dt = 1.0 / abf.dataRate  # Time step between samples (s)
time = np.arange(len(signal)) * dt

# ==== Spike detection ====
threshold = -35  # mV
spike_indices, _ = find_peaks(signal, height=threshold)
spike_times = time[spike_indices]

# ==== Burst detection ====
isi = np.diff(spike_times)
burst_threshold = 0.3  # s, max ISI within a burst

bursts = []
current_burst = [spike_times[0]]
for i in range(1, len(isi)):
    if isi[i-1] < burst_threshold:
        current_burst.append(spike_times[i])
    else:
        if len(current_burst) > 1:
            bursts.append(current_burst)
        current_burst = [spike_times[i]]
if len(current_burst) > 1:
    bursts.append(current_burst)

print(f'Total bursts detected: {len(bursts)}')

# ==== Plot first 10 bursts (one per figure) ====
n_bursts_to_plot = min(10, len(bursts))

for i in range(n_bursts_to_plot):
    burst = bursts[i]
    mask = (time >= burst[0]) & (time <= burst[-1])
    t_burst = time[mask]
    s_burst = signal[mask]

    burst_peak_mask = (spike_times >= burst[0]) & (spike_times <= burst[-1])
    burst_spike_times = spike_times[burst_peak_mask]
    burst_spike_values = signal[spike_indices][burst_peak_mask]

    plt.figure(figsize=(8, 3))
    plt.plot(t_burst, s_burst, lw=0.8, color='black')
    plt.plot(burst_spike_times, burst_spike_values, 'ro', markersize=3)
    plt.plot(burst_spike_times, burst_spike_values, 'r-', lw=1)

    plt.xlabel('Time (s)')
    plt.ylabel('Voltage (mV)')
    plt.title(f'Burst {i+1}')
    plt.tight_layout()
    plt.show()

In [ ]:
import pyabf
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.interpolate import interp1d

# ==== Load data ====
file_path = 'bursting/cell89basal.abf'
abf = pyabf.ABF(file_path)

# Concatenate sweeps
signal = np.concatenate([abf.setSweep(i) or abf.sweepY for i in range(abf.sweepCount)])
dt = 1.0 / abf.dataRate
time = np.arange(len(signal)) * dt

# ==== Spike detection ====
threshold = -35
spike_indices, _ = find_peaks(signal, height=threshold)
spike_times = time[spike_indices]

# ==== Burst detection ====
isi = np.diff(spike_times)
burst_threshold = 0.3

bursts = []
current_burst = [spike_times[0]]
for i in range(1, len(isi)):
    if isi[i-1] < burst_threshold:
        current_burst.append(spike_times[i])
    else:
        if len(current_burst) > 1:
            bursts.append(current_burst)
        current_burst = [spike_times[i]]
if len(current_burst) > 1:
    bursts.append(current_burst)

print(f'Total detected bursts: {len(bursts)}')

# ==== Normalization functions ====
def normalize_y(signal_segment):
    return (signal_segment - np.mean(signal_segment)) / np.std(signal_segment)

def rescale_x(time_segment, signal_segment, n_points=100):
    f = interp1d(np.linspace(0, 1, len(signal_segment)), signal_segment)
    return f(np.linspace(0, 1, n_points))

# ==== Plot first 10 normalized bursts ====
n_bursts_to_plot = min(10, len(bursts))
for i in range(n_bursts_to_plot):
    burst = bursts[i]
    mask = (time >= burst[0]) & (time <= burst[-1])
    t_burst = time[mask]
    s_burst = signal[mask]

    s_rescaled = rescale_x(t_burst, s_burst, n_points=100)
    s_normalized = normalize_y(s_rescaled)

    plt.figure(figsize=(6, 3))
    plt.plot(np.linspace(0, 1, 100), s_normalized, 'k-')
    plt.title(f'Burst {i+1} (normalized)')
    plt.xlabel('Time normalized')
    plt.ylabel('Normalized amplitude')
    plt.tight_layout()
    plt.show()

In [ ]:
import pyabf
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.interpolate import interp1d

# ==== Load data ====
file_path = 'bursting/cell89basal.abf'
abf = pyabf.ABF(file_path)

signal = np.concatenate([abf.setSweep(i) or abf.sweepY for i in range(abf.sweepCount)])
dt = 1.0 / abf.dataRate
time = np.arange(len(signal)) * dt

# ==== Spike detection ====
threshold = -35
spike_indices, _ = find_peaks(signal, height=threshold)
spike_times = time[spike_indices]

# ==== Burst detection ====
isi = np.diff(spike_times)
burst_threshold = 0.3

bursts = []
current_burst = [spike_times[0]]
for i in range(1, len(isi)):
    if isi[i-1] < burst_threshold:
        current_burst.append(spike_times[i])
    else:
        if len(current_burst) > 1:
            bursts.append((current_burst[0], current_burst[-1]))
        current_burst = [spike_times[i]]
if len(current_burst) > 1:
    bursts.append((current_burst[0], current_burst[-1]))

# ==== Classify bursts ====
square_wave_bursts = []
parabolic_bursts = []
other_bursts = []

for i, (burst_start, burst_end) in enumerate(bursts):
    burst_mask = (time >= burst_start) & (time <= burst_end)
    burst_min = np.min(signal[burst_mask])

    prev_mean = np.mean(signal[(time > bursts[i-1][1]) & (time < burst_start)]) if i > 0 else np.nan
    next_mean = np.mean(signal[(time > burst_end) & (time < bursts[i+1][0])]) if i < len(bursts)-1 else np.nan
    inter_mean = np.nanmean([prev_mean, next_mean])

    if burst_min > inter_mean:
        square_wave_bursts.append((burst_start, burst_end))
    elif burst_min < inter_mean:
        parabolic_bursts.append((burst_start, burst_end))
    else:
        other_bursts.append((burst_start, burst_end))

# ==== Normalization functions ====
def normalize_y(signal_segment):
    return (signal_segment - np.mean(signal_segment)) / np.std(signal_segment)

def rescale_x(time_segment, signal_segment, n_points=100):
    f = interp1d(np.linspace(0, 1, len(signal_segment)), signal_segment)
    return f(np.linspace(0, 1, n_points))

# ==== Overlay bursts ====
def overlay_bursts(burst_list, tipo, n=3):
    if len(burst_list) == 0:
        print(f'No bursts of type {tipo} to plot.')
        return
    n_to_plot = min(n, len(burst_list))
    plt.figure(figsize=(6, 3))
    for i in range(n_to_plot):
        start, end = burst_list[i]
        mask = (time >= start) & (time <= end)
        t_burst = time[mask]
        s_burst = signal[mask]

        s_rescaled = rescale_x(t_burst, s_burst, n_points=100)
        s_normalized = normalize_y(s_rescaled)

        plt.plot(np.linspace(0, 1, 100), s_normalized, lw=1, alpha=0.7, label=f'Burst {i+1}')

    plt.title(f'{tipo} Bursts Overlay (normalized)')
    plt.xlabel('Time normalized')
    plt.ylabel('Normalized amplitude')
    plt.legend()
    plt.tight_layout()
    plt.show()

overlay_bursts(square_wave_bursts, 'Square Wave', n=3)
overlay_bursts(parabolic_bursts, 'Parabolic', n=3)
overlay_bursts(other_bursts, 'Other', n=3)